<a href="https://colab.research.google.com/github/oluwafemiomitusa/Google-Earth-Engine_Urbanization/blob/main/Land_Use_Mapping_with_Google_Earth_Engine_Python_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries

import ee, geemap


# Trigger ee authentication
#ee.Authenticate()

# Initialize ee library
ee.Initialize()

In [2]:
# ROI
roi = ee.Geometry.Point([-96, 29.2])  # ROI is Houston
location = roi.centroid().coordinates().getInfo()[::-1]

# Load Landsat 8 data
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
.filterDate('2021-01-01', '2021-12-31') \
.filterBounds(roi) \
.sort('CLOUD_COVER') \
.first()

In [3]:
print(location)

[29.200000000000003, -96.00000000000001]


In [4]:
# Import ESRI Data
lc = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m").mosaic().clip(image.geometry())
label = 'b1'

#Sample the input imagery to get a FeatureCollection of training data.
sample = image.addBands(lc).sample(**{
    'region': image.geometry(),
    'numPixels': 1000,
    'seed': 0
})

# Add a random value field to the sample and use it to approximately split 80%
# of the features into a training set and 20% i to a validation set.
sample = sample.randomColumn()
trainingSample = sample.filter('random <= 0.8')
validationSample = sample.filter('random > 0.8')

# Train a 10-tree random forest classifier from the training sample.
trainedClassifier = ee.Classifier.smileRandomForest(10).train(**{
    'features': trainingSample,
    'classProperty': label,
    'inputProperties': bands
})

# Classify the reflectance image from the trained classifier.
model = image.classify(trainedClassifier)

In [5]:
# Import the Folium library
import folium

# Define a method for displaying Earth ENgine Image tiles to folium map.
def addLayer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name = name,
      overlay = True,
      control = True
      ).add_to(self)

# Create the map object
Map = folium.Map(location=location, zoom_start = 8)

# Add base map
basemaps = {'Google Satellite Hybrid': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Satellite Hybrid',
overlay = True,
control = True
)}
basemaps['Google Satellite Hybrid'].add_to(Map)
  

In [6]:
folium.Map.addLayer = addLayer

# Define a dictionary which will be used to make lgend and visualize image on map.
dict = {
    "names": [
        "Water",
        "Trees",
        "Grass",
        "Flooded Vegetation",
        "Crops",
        "Scrub/Shrub",
        "Built Area",
        "Bare Ground",
        "Snow/ice",
        "Clouds"
      ],

      "colors": [
          "#1A5BAB",
          "#358221",
          "#A7D282",
          "#87D19E",
          "#FFDB5C",
          "#EECFA8",
          "#ED022A",
          "#EDE9E4",
          "#F2FAFF",
          "#C8C8C8"
      ]}
# Add land cover to the image object.

#Map.addLayer(lc, {'min':1, 'max':10, 'pallete':dict['colors']}, 'ESRI LULC 10m')
Map.addLayer(model, {'min': 1, 'max': 10, 'palette': dict['colors']}, 'Classified 2021')


visParamsTrue = {'bands': ['B4', 'B3', 'B2'], 'min':0, 'max':3000, 'gamma':1.4}
Map.addLayer(image, visParamsTrue, 'Landsat 2021')

# Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

# Display the map.
display(Map)

# Add Color Bar to Map in Google Engine Python API

In [14]:
Map = geemap.Map()


# Load countries data
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");
roi = countries.filter(ee.Filter.eq("country_na","United States"));

dem = ee.Image("USGS/SRTMGL1_003").clip(roi)
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, vis_params, 'SRTM DEM')

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']
Map.add_colorbar(vis_params, label='Elevation (m)', layer_name="SRTM DEM") # Horizontal orientation
Map.centerObject(roi, 4)
Map

Map(center=[39.76850680995396, -99.05366228420321], controls=(WidgetControl(options=['position', 'transparent_…

In [8]:
# Vertical orientation
Map.add_colorbar(vis_params, label='Elevation (m)', layer_name="SRTM DEM", orientation="vertical")

In [16]:
# Add transparency to legend
Map.add_colorbar(vis_params, label='Elevation (m)', layer_name="SRTM DEM", orientation="vertical", transparent_bg=True)


In [18]:
# Add classification to legend
Map.add_colorbar(vis_params, label='Elevation (m)', layer_name="SRTM DEM", orientation="vertical", transparent_bg=True, discrete=True)
